# Testes como ferramenta para identificar código que pede refatoração

# Exercícios Práticos: Testes, Acoplamento e SOLID

**Objetivo:**
- Identificar problemas de acoplamento e violações de SOLID a partir de exemplos reais
- Perceber como essas questões afetam o design, testabilidade e manutenção do código
- Discutir e refatorar para aplicar boas práticas

**Como usar este notebook:**
1. Leia cada exemplo.
2. Tente prever quais problemas vão surgir ao testar.
3. Execute os testes e veja na prática as limitações.
4. Analise a versão refatorada e compare.
5. Discuta as perguntas sugeridas.

## Testes x Acoplamento
- **Dificuldade para testar = alerta de problema de design**
- Muitas dependências diretas, mocks e configurações complexas são indício de acoplamento alto
- Princípios SOLID ajudam a reduzir esses problemas

## Exercício 1: Acoplamento Excessivo
**ANTES (Problema):**

In [30]:
class Database:
    def save_order(self, customer_email, product_id):
        print(f"Salvou de verdade no banco a ordem para {customer_email}, {product_id}")

class EmailSender:
    def send_confirmation(self, customer_email):
        print(f"Email enviado de verdade para {customer_email}")

class OrderService:
    def create_order(self, customer_email, product_id):
        self.database = Database()
        self.database.save_order(customer_email, product_id)
        self.email_sender = EmailSender()
        self.email_sender.send_confirmation(customer_email)
        print("Order created!")
        return True

**Perguntas para discussão:**
- Por que é difícil testar essa classe isoladamente?
- Quais princípios SOLID estão sendo violados?
- Que tipo de dependências temos aqui?

In [31]:


class TestOrderService:
    def test_create_order(self):
        service = OrderService()
        assert service.create_order("joao@teste.com", 42), "Criação da Ordem falhou"

TestOrderService().test_create_order()

Salvou de verdade no banco a ordem para joao@teste.com, 42
Email enviado de verdade para joao@teste.com
Order created!


---
### Refatoração: Inversão de Dependência, Injeção de Dependências
**DEPOIS (bom):**

In [32]:
class OrderService:
    def __init__(self, database, email_sender):
        self.database = database
        self.email_sender = email_sender

    def create_order(self, customer_email, product_id):
        self.database.save_order(customer_email, product_id)
        self.email_sender.send_confirmation(customer_email)
        print("Ordem criada!")
        return True

In [33]:

# Testando a versão boa
class FakeDatabase:
    def save_order(self, customer_email, product_id):
        print(f"Faz de conta que salvou a ordem para {customer_email}, {product_id}")

class FakeEmailSender:
    def send_confirmation(self, customer_email):
        print(f"Faz de conta que enviou e-mail para {customer_email}")

class TestOrderServiceRefatorado:
    def test_create_order(self):
        db = FakeDatabase()
        email = FakeEmailSender()
        service = OrderService(db, email)
        assert service.create_order("joao@teste.com", 42), "Criar ordem falhou"


TestOrderServiceRefatorado().test_create_order()

Faz de conta que salvou a ordem para joao@teste.com, 42
Faz de conta que enviou e-mail para joao@teste.com
Ordem criada!


---
## Exercício 2: Princípio da Responsabilidade Única (SRP)
**ANTES (Problema):**

In [ ]:
class User:
    def __init__(self, name):
        self.name = name

    def save(self):
        print(f"Salvou o usuário {self.name} no banco de dados")
        print(f"Enviou e-mail de boas-vindas ao usuário {self.name}")
        return True


**Perguntas:**
- Por que pode ser ruim testar isso?
- O que acontece se mudar a regra de salvar ou enviar email?

In [ ]:
# Tentando testar

class TestUserSRP:
    
    def test_save_user(self):
        user = User("Joana")
        assert user.save(), "Salvar usuário falhou"

        
    def test_email_user(self):
        user = User("Joana")
        assert user.save(), "Enviar e-mail falhou"
        
TestUserSRP().test_save_user()
TestUserSRP().test_email_user()

Os testes ficaram iguais... 
Como eu testo apenas o salvamento ou apenas o envio do e-mail?
Se um dos dois falhar como eu sei que o outro não falhou também?

**DEPOIS (SRP aplicado):**

In [ ]:
class User:
    def __init__(self, name):
        self.name = name

class UserRepository:
    def save(self, user):
        print(f"Salvou o usuário {self.name} no banco de dados")
        return True

class EmailSender:
    def send_welcome_email(self, user):
        print(f"Enviou e-mail de boas-vindas ao usuário {self.name}")
        return True

In [ ]:
# Testando a versão boa do SRP
class FakeRepo:
    def save(self, user): 
        print("Salvou de mentirinha!")
        return True

class FakeEmailSender:
    def send_welcome_email(self, user): 
        print("Enviou e-mail de mentirinha!")
        return True

class TestUserSRP:
    def test_save(self):
        user = User("Joana")
        repo = FakeRepo()
        assert repo.save(user), "Salvamento falhou"

    def test_send_email(self):
        user = User("Joana")
        email = FakeEmailSender()
        assert email.send_welcome_email(user), "Envio de e-mail falhou"
        
TestUserSRP().test_save()
TestUserSRP().test_send_email()

---
## Exercício 3: Violação do DIP (Princípio da Inversão de Dependência)
**ANTES (Problema):**

In [ ]:
class FileWriter:
    def write(self, report):
        print(f"Writing report: {report}")

class ReportGenerator:
    def __init__(self):
        self.file_writer = FileWriter()

    def generate(self, data):
        report = self._build_report(data)
        self.file_writer.write(report)

    def _build_report(self, data):
        return "report: " + str(data)

**Teste Complexo**

In [ ]:
class TestReportGenerator:
    def test_generate_calls_write(self):
        class MockWriter:
            def __init__(self):
                self.called = False
                self.last_report = None
            def write(self, report):
                self.called = True
                self.last_report = report

        writer = MockWriter()
        rg = ReportGenerator()
        rg.file_writer = writer  # Sobrescreve a dependencia para conseguir testar
        rg.generate({"foo": "bar"})
        assert writer.called, "Não chamou o file writer"
        assert writer.last_report == "report: {'foo': 'bar'}", "Relatório diferente"

TestReportGenerator().test_generate_calls_write()

**Perguntas:**
- O que acontece se quiser mudar de FileWriter para outro tipo de saída?
- O que dificulta no teste?

**DEPOIS (DIP aplicado):**

In [ ]:
from typing import Protocol


class Writer(Protocol):
    
    def write(self, report: str):
        pass


class ReportGenerator:
    def __init__(self, writer: Writer):
        self.writer = writer

    def generate(self, data):
        report = self._build_report(data)
        self.writer.write(report)

    def _build_report(self, data):
        return "report: " + str(data)

In [ ]:
class FakeWriter(Writer):
    def __init__(self): self.written = False
    def write(self, report): self.written = True

class TestReportGeneratorDIP:
    def test_generate(self):
        writer = FakeWriter()
        rg = ReportGenerator(writer)
        rg.generate({"id": 1, "value": 10})
        assert writer.written

TestReportGeneratorDIP().test_generate()

---
## Exercício 4: God Object
**ANTES (Problema):**

In [ ]:
class GodObject:
    def __init__(self, calculadora_salario, order_service, invoice_service, email_service) -> None:
        pass
    def calculate_salary(self): pass
    def process_order(self): pass
    def generate_invoice(self): pass
    def send_email(self): pass

**Perguntas:**
- Quais problemas surgem ao testar uma classe assim?
- Como identificar um God Object?

In [37]:
from typing import Protocol

class ProcessOrdem(Protocol):
    def process_order(self): pass
    
class ProcessarOrdemUseCase:
    
    def __init__(self, processador: ProcessOrdem):
        self.processador = processador
        
        
    def execute(self):
        self.processador.process_order()

In [39]:
usecase = ProcessarOrdemUseCase(processador=GodObject())
usecase.execute()

In [35]:
import unittest

class TestGodObject(unittest.TestCase):
    def test_calculate_salary(self):
        god = GodObject()
        self.assertIsNone(god.calculate_salary())

    def test_process_order(self):
        god = GodObject()
        self.assertIsNone(god.process_order())

    def test_generate_invoice(self):
        god = GodObject()
        self.assertIsNone(god.generate_invoice())

    def test_send_email(self):
        god = GodObject()
        self.assertIsNone(god.send_email())

unittest.main(argv=[''], exit=False)

E.......
ERROR: test_send (__main__.TestEmailSender.test_send)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/cq/7tzwypmd78xbdrrzjtd5xcrw0000gn/T/ipykernel_88999/1747250128.py", line 19, in test_send
    self.assertIsNone(sender.send())
                      ^^^^^^^^^^^
AttributeError: 'EmailSender' object has no attribute 'send'

----------------------------------------------------------------------
Ran 8 tests in 0.004s

FAILED (errors=1)


**DEPOIS (responsabilidades separadas):**

In [ ]:
class SalaryCalculator:
    def __init__(self, dep1): pass
    def calculate(self): pass

class OrderProcessor:
    def __init__(self, dep2): pass
    def process(self): pass

class InvoiceGenerator:
    def __init__(self, dep3): pass
    def generate(self): pass

class EmailSender:
    def __init__(self, dep4): pass
    def send(self): pass

In [ ]:
from unittest.mock import Mock


class TestSalaryCalculator(unittest.TestCase):
    def test_calculate(self):
        calc = SalaryCalculator(Mock(spec="dep1"))
        self.assertIsNone(calc.calculate())

class TestOrderProcessor(unittest.TestCase):
    def test_process(self):
        processor = OrderProcessor(Mock(spec="dep2"))
        self.assertIsNone(processor.process())

class TestInvoiceGenerator(unittest.TestCase):
    def test_generate(self):
        generator = InvoiceGenerator(Mock(spec="dep3"))
        self.assertIsNone(generator.generate())

class TestEmailSender(unittest.TestCase):
    def test_send(self):
        sender = EmailSender(Mock(spec="dep4"))
        self.assertIsNone(sender.send())

unittest.main(argv=[''], exit=False)

---
## Para Discussão em Grupo
- Em qual exercício você sentiu mais diferença na hora de testar?
- Que sinais de 'código difícil de testar' você encontrou?
- Como os princípios SOLID facilitam testes automatizados?
- Você já viu casos parecidos no seu dia a dia?

_Notebook preparado para a Guilda de Engenharia de Software – para facilitar discussão e prática sobre testes, acoplamento e SOLID._

# The End

Obrigado.